In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
from tqdm import tqdm
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
# ## initialize ssh_compute
# ssh_c = server.ssh_interface(
#     nbytes_toReceive=20000,
#     recv_timeout=1,
#     verbose=True,
# )
# ssh_c.o2_connect(
#     hostname=remote_host_compute,
#     username=username,
#     password=server.pw_decode(pw),
#     key_filename=path_sshKey,
#     look_for_keys=False,
#     passcode_method=1,
#     verbose=0,
#     skip_passcode=False,    
# )

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
# send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Download stat and ops files

In [13]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/',
    search_pattern_re='stat.*npy',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230804/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230726/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230708/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230808/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230711/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230725/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230721/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230

In [14]:
paths_found = [p for p in paths_found if 'exp' in p]

In [15]:
dates_toUse = natsort.natsorted([Path(p).parts[-7] for p in paths_found])
dates_toUse

['20230702',
 '20230703',
 '20230705',
 '20230706',
 '20230707',
 '20230708',
 '20230709',
 '20230710',
 '20230711',
 '20230712',
 '20230713',
 '20230714',
 '20230715',
 '20230716',
 '20230717',
 '20230718',
 '20230719',
 '20230720',
 '20230721',
 '20230722',
 '20230723',
 '20230724',
 '20230725',
 '20230726',
 '20230727',
 '20230728',
 '20230729',
 '20230730',
 '20230731',
 '20230801',
 '20230802',
 '20230803',
 '20230804',
 '20230805',
 '20230806',
 '20230807',
 '20230808']

In [16]:
[sftp.get(str(Path(p).parent / 'stat.npy'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/stat_and_ops') / Path(p).parts[-7] / 'stat.npy')) for p in paths_found[2:]];

100%|█████████████████████████████████████▉| 26.6M/26.6M [00:00<00:00, 41.4Mb/s]
100%|█████████████████████████████████████▉| 26.4M/26.4M [00:00<00:00, 47.5Mb/s]
100%|█████████████████████████████████████▉| 25.8M/25.8M [00:00<00:00, 50.2Mb/s]
100%|█████████████████████████████████████▉| 25.7M/25.7M [00:00<00:00, 45.5Mb/s]
100%|█████████████████████████████████████▉| 26.2M/26.2M [00:00<00:00, 47.7Mb/s]
100%|█████████████████████████████████████▉| 26.2M/26.2M [00:00<00:00, 40.8Mb/s]
100%|█████████████████████████████████████▉| 25.7M/25.8M [00:00<00:00, 46.1Mb/s]
100%|█████████████████████████████████████▉| 25.7M/25.7M [00:00<00:00, 48.4Mb/s]
100%|█████████████████████████████████████▉| 25.9M/25.9M [00:00<00:00, 49.7Mb/s]
100%|█████████████████████████████████████▉| 26.4M/26.5M [00:00<00:00, 50.7Mb/s]
100%|█████████████████████████████████████▉| 26.7M/26.7M [00:00<00:00, 52.1Mb/s]
100%|█████████████████████████████████████▉| 26.2M/26.3M [00:00<00:00, 51.0Mb/s]
100%|███████████████████████

In [17]:
[sftp.get(str(Path(p).parent / 'ops.npy'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/stat_and_ops') / Path(p).parts[-7] / 'ops.npy')) for p in paths_found[2:]];

100%|███████████████████████████████████████▉| 231M/231M [00:03<00:00, 60.8Mb/s]
100%|███████████████████████████████████████▉| 231M/231M [00:05<00:00, 42.6Mb/s]
100%|███████████████████████████████████████▉| 231M/231M [00:03<00:00, 57.8Mb/s]
100%|███████████████████████████████████████▉| 231M/231M [00:04<00:00, 54.7Mb/s]
100%|███████████████████████████████████████▉| 231M/231M [00:05<00:00, 41.6Mb/s]
100%|███████████████████████████████████████▉| 231M/231M [00:04<00:00, 56.9Mb/s]
100%|███████████████████████████████████████▉| 230M/230M [00:04<00:00, 56.7Mb/s]
100%|███████████████████████████████████████▉| 230M/230M [00:04<00:00, 54.1Mb/s]
100%|███████████████████████████████████████▉| 229M/229M [00:04<00:00, 57.2Mb/s]
100%|███████████████████████████████████████▉| 230M/230M [00:04<00:00, 56.7Mb/s]
100%|███████████████████████████████████████▉| 230M/230M [00:04<00:00, 56.6Mb/s]
100%|███████████████████████████████████████▉| 228M/228M [00:03<00:00, 57.3Mb/s]
100%|███████████████████████

# Download logger files

In [18]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/',
    search_pattern_re='logger\.mat',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230804/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230810/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230726/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230708/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230808/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230711/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230725/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230721/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230722/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230717/analysis_data/logger.mat
/n/files/Neurobio/MI

In [19]:
dates_toUse = natsort.natsorted([Path(p).parts[-3] for p in paths_found])
dates_toUse

['20230702',
 '20230703',
 '20230705',
 '20230706',
 '20230707',
 '20230708',
 '20230709',
 '20230710',
 '20230711',
 '20230712',
 '20230713',
 '20230714',
 '20230715',
 '20230716',
 '20230717',
 '20230718',
 '20230719',
 '20230720',
 '20230721',
 '20230722',
 '20230723',
 '20230724',
 '20230725',
 '20230726',
 '20230727',
 '20230728',
 '20230729',
 '20230730',
 '20230731',
 '20230801',
 '20230802',
 '20230803',
 '20230804',
 '20230805',
 '20230806',
 '20230807',
 '20230808',
 '20230809',
 '20230810',
 '20230811']

In [20]:
[sftp.get(str(Path(p).parent / 'logger.mat'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/loggers') / date / 'logger.mat')) for date, p in zip(dates_toUse, paths_found)];

100%|█████████████████████████████████████▉| 4.82M/4.82M [00:00<00:00, 7.40Mb/s]
100%|█████████████████████████████████████▉| 4.82M/4.82M [00:00<00:00, 6.43Mb/s]
100%|█████████████████████████████████████▉| 13.4M/13.5M [00:02<00:00, 5.51Mb/s]
100%|█████████████████████████████████████▉| 13.5M/13.5M [00:08<00:00, 1.60Mb/s]
100%|██████████████████████████████████████▉| 13.5M/13.5M [00:15<00:00, 896kb/s]
100%|█████████████████████████████████████▉| 12.2M/12.2M [00:09<00:00, 1.26Mb/s]
100%|█████████████████████████████████████▉| 11.3M/11.3M [00:10<00:00, 1.05Mb/s]
100%|█████████████████████████████████████▉| 8.65M/8.67M [00:07<00:00, 1.18Mb/s]
100%|█████████████████████████████████████▉| 10.3M/10.3M [00:00<00:00, 27.7Mb/s]
100%|█████████████████████████████████████▉| 11.8M/11.8M [00:00<00:00, 13.0Mb/s]
100%|█████████████████████████████████████▉| 13.5M/13.6M [00:00<00:00, 33.7Mb/s]
100%|█████████████████████████████████████▉| 13.6M/13.6M [00:00<00:00, 31.7Mb/s]
100%|███████████████████████

In [21]:
[sftp.get(str(Path(p).parent / 'expParams.mat'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/loggers') / date / 'expParams.mat')) for date, p in zip(dates_toUse, paths_found)];

 95%|█████████████████████████████████████▏ | 65.5k/68.6k [00:00<00:00, 985kb/s]
 95%|█████████████████████████████████████▏ | 65.5k/68.6k [00:00<00:00, 590kb/s]
100%|█████████████████████████████████████▊| 9.08M/9.11M [00:00<00:00, 27.1Mb/s]
100%|█████████████████████████████████████▉| 9.24M/9.27M [00:00<00:00, 29.1Mb/s]
100%|█████████████████████████████████████▉| 8.68M/8.70M [00:02<00:00, 3.36Mb/s]
100%|█████████████████████████████████████▉| 9.11M/9.11M [00:00<00:00, 28.0Mb/s]
100%|█████████████████████████████████████▉| 9.01M/9.02M [00:04<00:00, 2.23Mb/s]
100%|█████████████████████████████████████▉| 8.91M/8.91M [00:01<00:00, 5.61Mb/s]
100%|█████████████████████████████████████▊| 8.59M/8.62M [00:00<00:00, 9.72Mb/s]
100%|█████████████████████████████████████▉| 8.75M/8.76M [00:00<00:00, 29.6Mb/s]
100%|█████████████████████████████████████▉| 8.78M/8.78M [00:00<00:00, 26.1Mb/s]
100%|█████████████████████████████████████▉| 9.01M/9.03M [00:00<00:00, 28.2Mb/s]
100%|███████████████████████

# Download wavesurfer files

In [22]:
paths_files_remote = sftp.search_recursive(
    path=r'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/',
    search_pattern_re=r'.*exp.*h5',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_files_remote = natsort.natsorted(paths_files_remote)

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230804/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230810/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230726/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230708/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230808/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230711/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230725/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230721/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230722/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230717/wavesurfer_data/ex

In [23]:
dates_files_remote = [Path(p).parent.parent.name for p in paths_files_remote]

In [24]:
dates_files_remote

['20230702',
 '20230703',
 '20230705',
 '20230706',
 '20230707',
 '20230708',
 '20230709',
 '20230710',
 '20230711',
 '20230712',
 '20230713',
 '20230714',
 '20230715',
 '20230716',
 '20230717',
 '20230718',
 '20230719',
 '20230720',
 '20230721',
 '20230722',
 '20230723',
 '20230724',
 '20230725',
 '20230726',
 '20230727',
 '20230728',
 '20230729',
 '20230730',
 '20230731',
 '20230801',
 '20230802',
 '20230803',
 '20230804',
 '20230805',
 '20230806',
 '20230807',
 '20230808',
 '20230809',
 '20230810',
 '20230811']

In [25]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/wavesurfer_files/'

In [26]:
[sftp.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_files_remote, dates_files_remote)]

100%|█████████████████████████████████████▉| 50.7M/50.7M [00:04<00:00, 12.4Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 28.4Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 31.9Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 37.9Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 26.4Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:05<00:00, 8.49Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:05<00:00, 9.19Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 29.3Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 26.7Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 29.7Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:01<00:00, 28.6Mb/s]
100%|█████████████████████████████████████▉| 50.7M/50.7M [00:02<00:00, 23.8Mb/s]
100%|███████████████████████

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Download camera CSV files

In [27]:
paths_csvs_remote = sftp.search_recursive(
    path=r'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/',
    search_pattern_re=r'.*cam4.*csv',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_csvs_remote = natsort.natsorted(paths_csvs_remote)

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230804/camera_data/exp/cam4/times_cam42023-08-04T12_06_21.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230726/camera_data/exp/cam4/times_cam42023-07-26T11_49_44.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230708/camera_data/exp/cam4/times_cam42023-07-08T10_28_33.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230808/camera_data/exp/cam4/times_cam42023-08-08T13_36_08.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230711/camera_data/exp/cam4/times_cam42023-07-11T10_48_34.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230725/camera_data/exp/cam4/times_cam42023-07-25T11_19_04.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230721/camera_data/exp/cam4/times_cam42023-07-21T11_21_03.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403L/20230722/camera_data/exp/cam4/times_cam4

In [28]:
dates_csvs_remote = [Path(p).parent.parent.parent.parent.name for p in paths_csvs_remote]

In [29]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/cam4_CSVs/'

In [30]:
[sftp.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_csvs_remote, dates_csvs_remote)];

100%|█████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 44.8Mb/s]
100%|█████████████████████████████████████▉| 31.2M/31.2M [00:00<00:00, 48.7Mb/s]
100%|█████████████████████████████████████▉| 33.1M/33.2M [00:00<00:00, 52.5Mb/s]
100%|█████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 46.9Mb/s]
100%|█████████████████████████████████████▉| 31.1M/31.2M [00:00<00:00, 39.0Mb/s]
100%|█████████████████████████████████████▉| 31.2M/31.2M [00:00<00:00, 37.6Mb/s]
100%|█████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 47.9Mb/s]
100%|█████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 44.7Mb/s]
100%|█████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 45.5Mb/s]
100%|█████████████████████████████████████▉| 31.2M/31.2M [00:00<00:00, 45.4Mb/s]
100%|█████████████████████████████████████▉| 31.0M/31.1M [00:00<00:00, 44.0Mb/s]
100%|█████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 46.1Mb/s]
100%|███████████████████████

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Download face-rhythm files

In [11]:
paths_fr_remote = natsort.natsorted(sftp.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701',
    search_pattern_re=r'VQT_Analyzer.h5',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230531/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230420/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230523/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230517/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230515/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230430/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230512/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230511/

In [12]:
dates_fr_remote = [Path(p).parts[-4] for p in paths_fr_remote]
dates_fr_remote

['20230420',
 '20230424',
 '20230425',
 '20230426',
 '20230427',
 '20230428',
 '20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230510_badExpDay',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230530',
 '20230531']

In [14]:
dirsToDownload_fr_remote = [str(Path(p).parent.parent.parent) for p in paths_fr_remote]
dirsToDownload_fr_remote

['/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230420',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230424',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230425',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230426',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230427',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230428',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230430',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230501',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230502',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230503',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mo

In [15]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/fr_run'

In [ ]:
directory_local_save

In [ ]:
sftp.get_dir(source=Path(p_r).parent.parent.parent.parent, target=directory_local_save)

# Download dFoF and tqm files

In [12]:
dir_dFoF_and_tqm_remote = r'/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R'
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/'

In [13]:
sftp.get_dir(source=dir_dFoF_and_tqm_remote, target=directory_local_save)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R/20230531/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/20230531/parameters_batch.json


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R/20230531/jobNum_0/tqm.pkl   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/20230531/jobNum_0/tqm.pkl


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R/20230531/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/20230531/jobNum_0/params.json


...

0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R/20230502/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/20230502/dispatcher.py


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R/20230502/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/20230502/notes.txt


0.00b [00:00, ?b/s]


In [ ]:
paths_dFoF_remote = natsort.natsorted(sftp.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0322R',
    search_pattern_re=r'dFoF.npy',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
))

In [42]:
dates_dFoF_remote = [Path(p).parts[-3] for p in paths_dFoF_remote]

In [43]:
dates_dFoF_remote

['20230419',
 '20230420',
 '20230424',
 '20230425',
 '20230426',
 '20230427',
 '20230428',
 '20230429',
 '20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230530',
 '20230531']

In [44]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/dFoF_and_tqm/'

In [ ]:
[sftp.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_csvs_remote, dates_dFoF_remote)];

In [ ]:
[sftp.get(remotepath=str(Path(p_r).parent / 'iscell_tqm.npy'), localpath=str(Path(directory_local_save) / date / 'iscell_tqm.npy')) for p_r, date in zip(paths_csvs_remote, dates_dFoF_remote)];

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()